# Regresión Lineal

## El paquete Scikit-Learn

El paquete Scikit-Learn es un conjunto de librerías que implementan distintos algoritmos de aprendizaje automático específico para Python.

In [1]:
from IPython.display import IFrame
IFrame('http://scikit-learn.org/stable/', width='100%', height=350)

Para el ejemplo que hemos estado trabajando, vamos a utilizar la librería de **eliminación recursiva de características** o *recursive features elimination*, `RFE`. Esta librería realiza una selección hacia atrás de variables eliminando cada vez una variable hasta llegar al número de variables especifícado desde el comienzo.

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


In [4]:
data = pd.read_csv("Advertising.csv")

In [5]:
feature_cols = ["TV", "Radio", "Newspaper"]

In [6]:
x = data[feature_cols]
y = data["Sales"]

In [7]:
estimator = SVR(kernel = "linear") # para la regresión lineal
selector = RFE(estimator,2,step=1) # se desea un modelo con dos variables predictoras
selector = selector.fit(x,y)       # especifica los datos para x, y

In [8]:
selector.support_

array([ True,  True, False])

In [11]:
selector.support_?

Este resultado informa sobre la selección que el algoritmo ha hecho de las variables predictivas más significativas.

In [12]:
selector.ranking_

array([1, 1, 2])

Calculemos la regresión lineal multivariable por medio de Scikit-learn utilizando la librería `Linear Regression`

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
x_predict = x[["TV" , "Radio"]]

In [15]:
lm = LinearRegression()
lm.fit(x_predict, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
lm.intercept_

2.921099912405138

In [17]:
lm.coef_

array([0.04575482, 0.18799423])

In [18]:
# factor R²
lm.score(x_predict, y)

0.8971942610828956

In [9]:
lm.score?

Object `lm.score` not found.


## Predicción usando variables categóricas

En general, una **variable categórica** se refiere a una variable que define cierta clase o categoría y toma con frecuencia valores discretos para representar cada clase. 

Un método para utilizar una variable categóricas como variable predictiva en una regresión lineal es descomponiendo la variable categórica en subvariables que representen cada clase.

Seas el modelo de regresión lineal multivariable: $$ y = \alpha + \beta_1 x_1, \beta_2 x_2, ... , \beta_n x_n + \varepsilon $$ donde alguna $x_k$ es categórica, entonces se descompone la variable $x_k$ en tantas variables auxiliares que representen cierta categoría(s) a considerar dentro del modelo.

**Ejemplo 1:**

Sea $x_k$ una variable categórica que toma el valor $1$ para representar el género masculino y $0$ para representar al femenino. El modelo lineal multivariable para estimar la salida considerando el género como variable predictora será alguno de los siguientes:

* Para considerar la categoría hombre: $$ y = \alpha + \beta_1 x_1, \beta_2 x_2, ...,\beta_k x_k , ..., \beta_n x_n + \varepsilon $$ 
* Para considerar la categoría mujer: $$ y = \alpha + \beta_1 x_1, \beta_2 x_2, ..., \beta_n x_n + \varepsilon $$

El modelo tendrá $x_k$ sólo cuando se considere al género masculino. 


**Ejemplo 2:**

Sea $x_k$ una variable categórica que toma el valor $x_{k1} = 1$ para representar la ciudad 1, $x_{k2} = 1$ para representar la ciudad 2, y $x_{k3} = 3$ para representar la ciudad 3. El modelo lineal multivariable para estimar la salida considerando una ciudad específica como variable predictora será alguno de los siguientes:

* Para considerar la categoría "ciudad 1":
    * $x_{k1} = 1$ si pertenece a la ciudad 1 y $x_{k1} = 0$ si no pertenece a la ciudad 1.
    * $y = \alpha + \beta_1 x_1, \beta_2 x_3, ...,\beta_k x_{k1} , ..., \beta_n x_n + \varepsilon $
    
* Para considerar la categoría "ciudad 2":
    * $x_{k2} = 1$ si pertenece a la ciudad 2 y $x_{k2} = 0$ si no pertenece a la ciudad 2.
    * $y = \alpha + \beta_1 x_1, \beta_2 x_3, ...,\beta_k x_{k2} , ..., \beta_n x_n + \varepsilon $ 
    
* Para considerar la categoría "ciudad 3":
    * No se pertenece a la ciudad 1 ni a la ciudad 2.
    * $y = \alpha + \beta_1 x_1, \beta_2 x_3, ..., \beta_n x_n + \varepsilon $

Al considerar una variable categórica $x_k$ como variable predictora, se requerirá agregar las variable $x_{kj}$ para $j \in \{1,..,c-1\}$ donde $c$ son las categorías. Con esto se predice la salida incluyendo una categoria $x_{kj}$ específica de la variable categórica considerada. 

**Nota:** la última categoría no necesita variable.


### Ejercicio

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [10]:
data = pd.read_csv("Ecom Expense.csv")

In [11]:
data.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


In [12]:
dummy_gender = pd.get_dummies(data['Gender'], prefix = 'Gender')
dummy_city_tier = pd.get_dummies(data['City Tier'], prefix = 'City')

In [13]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [14]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [15]:
# crea lista de columnas
column_names = data.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [16]:
data_new = data[column_names].join(dummy_gender)
data_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


In [17]:
# crea nueva lista de columnas
column_names = data_new.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend',
 'Gender_Female',
 'Gender_Male']

In [18]:
data_new = data_new[column_names].join(dummy_city_tier)
data_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


In [19]:
column_names = data_new.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend',
 'Gender_Female',
 'Gender_Male',
 'City_Tier 1',
 'City_Tier 2',
 'City_Tier 3']

In [20]:
feature_cols = ['Monthly Income', 'Transaction Time',
                 'Gender_Female', 'Gender_Male',
                'City_Tier 1', 'City_Tier 2', 'City_Tier 3']

In [21]:
x = data_new[feature_cols]
y = data_new["Total Spend"]

In [22]:
lm = LinearRegression()
lm.fit(x,y)

NameError: name 'LinearRegression' is not defined

In [34]:
lm.intercept_

3655.729407690652

In [35]:
lm.coef_

array([   0.15297825,    0.12372609,  -94.15779883,   94.15779883,
        119.6632516 ,  -16.67901801, -102.9842336 ])

In [36]:
list(zip(feature_cols, lm.coef_)) # zip compacta dos arreglos

[('Monthly Income', 0.1529782460932052),
 ('Transaction Time', 0.12372608642619978),
 ('Gender_Female', -94.1577988303201),
 ('Gender_Male', 94.15779883032008),
 ('City_Tier 1', 119.66325160390102),
 ('City_Tier 2', -16.67901800799043),
 ('City_Tier 3', -102.98423359591067)]

In [37]:
# factor R² 
lm.score(x,y)

0.19478920552885381

Con las variables predictoras seleccionadas el modelo lineal obtenido tiene muy mal ajuste.

Probemos utilizando nuevas variables predictoras, por ejemplo consideremos la variable `Record`

In [38]:
feature_cols = ['Monthly Income', 'Transaction Time',
                 'Gender_Female', 'Gender_Male',
                'City_Tier 1', 'City_Tier 2', 'City_Tier 3',
                'Record']

In [39]:
x = data_new[feature_cols]
y = data_new["Total Spend"]

In [40]:
lm = LinearRegression()
lm.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
lm.intercept_

-79.4171303013718

In [42]:
lm.coef_

array([ 1.47538980e-01,  1.54946125e-01, -1.31025013e+02,  1.31025013e+02,
        7.67643260e+01,  5.51389743e+01, -1.31903300e+02,  7.72233446e+02])

In [43]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589615),
 ('Gender_Female', -131.02501325554607),
 ('Gender_Male', 131.02501325554613),
 ('City_Tier 1', 76.76432601049524),
 ('City_Tier 2', 55.138974309232566),
 ('City_Tier 3', -131.9033003197279),
 ('Record', 772.2334457445644)]

In [44]:
# factor R² 
lm.score(x,y)

0.9179923586131016

Con las variables predictoras seleccionadas anteriormente y añadiendo la variable predictora *'Record'* el modelo lineal que se obtiene mejora bastante el ajuste.

El modelo queda entonces como:

***Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime - 131.02GenFemale + 131.02GenMale + 76.76CityTier1 + 55.3CityTier2 - 131.90CityTier3 + 772.23Record***

In [57]:
data_new['Prediction'] = -79.4171303013718 + data_new['Monthly Income'] * 0.14753898049205744 + data_new['Transaction Time'] * 0.15494612549589615 + data_new['Gender_Female'] * -131.02501325554607 + data_new['Gender_Male'] * 131.02501325554613 + data_new['City_Tier 1'] * 76.76432601049524 + data_new['City_Tier 2'] * 55.138974309232566 + data_new['City_Tier 3'] * -131.9033003197279 + data_new['Record'] * 772.2334457445644

In [48]:
data_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,Prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


In [51]:
SSD = np.sum((data_new['Prediction'] - data_new['Total Spend']) ** 2)
SSD

1517733985.3408163

In [53]:
RSE = np.sqrt(SSD/(len(data_new)-len(feature_cols)-1))
RSE

803.1318809818165

In [55]:
sales_mean = np.mean(data_new['Total Spend'])
sales_mean

6163.176415976714

In [56]:
error = RSE/sales_mean
error * 100

13.031135680294161

Es importante mencionar que el modelo predictivo anterior define 6 modelos diferentes conforme a los valores que se van a considerar en las variables categóricas analizadas. Esto es:
* Si es hombre y vive en CT1:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime -131.02(0) + 131.02(1) + 76.76(1) + 55.3(0) - 131.90(0) +  772.23Record* 

* Si es hombre y vive en CT2:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime -131.02(0) + 131.02(1) + 77.76(0) + 55.30(1) - 131.90(0) + 772.23Record*

* Si es hombre y vive en CT3:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime -131.02(0)+ 131.02(1) + 76.76(0) + 55.30(0) - 131.90 + 772.23Record*

* Si es mujer y vive en CT1:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime - 131.02(1) + 131.02(0) + 76.76(1) +  55.3(0) - 131.90(0) + 772.23Record*



* Si es mujer y vive en CT2:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime - 131.02(1) + 131.02(0) + 76.76(0) +  55.3(1) - 131.90(0) + 772.23Record*

* Si es mujer y vive en CT3:

*Total Spend = -79.41 + 0.14MontlyIncome + 0.15TransTime - 131.02(1) + 131.02(0) + 76.76(0) +  55.3(0) - 131.90(1) + 772.23Record*


Ahora bien, notemos que en los modelos anteriores se pueden sumar los coeficientes independientes lo cual reduce cada uno de los modelos.

Por otra parte es importante resaltar que el tratamiento de variables categóricas por medio de la inclusión de variables dummy $x_{kj}$ considera a $j = 1...c-1$ (donde $c$ es el número de categorias para cada variable categórica $k$), esto es que el número de variables dummy por cada variable categórica es una menos que el número de categorías.

Utilizando estas consideraciones, se puede reducir el número de modelos predictivos resultantes y la cantidad de variables predictoras que aparecen en ellos. Entonces podemos decir que en el ejemplo anterior se utilizaron variables dummy redundantes las cuales agregaron mayor información al modelo.

A continuación se muestra el manejo de variables categóricas por medio de variables dummy no redundantes, a este tratamiento se conoce como **enmascaramiento de varialbles**. 

## Eliminación de variables dummy redundantes

In [23]:
data

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485
5,TXN006,49,6,6282,48.974268,2,Male,Tier 2,2375.036467
6,TXN007,21,14,7086,961.203768,8,Male,Tier 1,7494.474559
7,TXN008,58,9,8881,962.253740,10,Male,Tier 3,10782.944920
8,TXN009,20,6,5635,858.328131,5,Male,Tier 1,3854.277411
9,TXN010,48,12,20861,43.036737,4,Female,Tier 2,5346.140262


In [25]:
dummy_gender = pd.get_dummies(data['Gender'], prefix = 'Gender').iloc[:,1:] # elige la segunda columna en adelante
dummy_gender.head()

,Gender_Male
0,0
1,0
2,1
3,0
4,0


In [26]:
dummy_city_tier = pd.get_dummies(data['City Tier'], prefix = 'City').iloc[:,1:] # elige la segunda columna en adelante
dummy_city_tier.head()

,City_Tier 2,City_Tier 3
0,0,0
1,1,0
2,1,0
3,0,0
4,1,0


In [27]:
column_names = data.columns.values.tolist()
data_new = data[column_names].join(dummy_gender)
data_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,0


In [101]:
column_names = data_new.columns.values.tolist()
data_new = data_new[column_names].join(dummy_city_tier)
data_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,0,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,0,1,0


In [106]:
feature_cols = ['Monthly Income', 'Transaction Time', 'Gender_Male', 'City_Tier 2', 'City_Tier 3','Record']
x = data_new[feature_cols]
y = data_new['Total Spend']
lm = LinearRegression()
lm.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [107]:
print(lm.intercept_)

-133.67781754642238


In [108]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589837),
 ('Gender_Male', 262.050026511094),
 ('City_Tier 2', -21.625351701262645),
 ('City_Tier 3', -208.66762633022344),
 ('Record', 772.2334457445639)]

In [105]:
lm.score(x,y)

0.19478920552885381

El número de varialbes predictoras se redujo sin que esto disminuya su significancia, el modelo sigue teniendo buen ajuste. 

Los coeficientes obtenidos para ambos modelos son los siguientes:

| variable (M1)  |  coeficiente (M1)   |  variable (M2)  |  coeficiente (M2)  |
|----------------|---------------------|---------------- |--------------------|
| Intercept      | -79.4171303013718   | Intercept       |-133.67781754642238 |
|Monthly Income  | 0.14753898049205744 | Monthly Income  |0.14753898049205744 | 
|Transaction Time| 0.15494612549589615 | Transaction Time|0.15494612549589837 |
|Gender_Female   | -131.02501325554607 |                                      |
|Gender_Male     | 131.02501325554613  | Gender_Male     |262.050026511094    |
|City_Tier 1     | 76.76432601049524   |                                      |
|City_Tier 2     | 55.138974309232566  | City_Tier 2     |-21.625351701262645 |
|City_Tier 3     | -131.9033003197279  | City_Tier 3     |-208.66762633022344 |
|Record          | 772.2334457445644   | Record          |772.2334457445639   |


Notemos que el valor de:
* `Gender Male` del modelo 2 es: 262.050026511094 = 131.02501325554613 - (-131.02501325554607) del modelo 1
* `City_Tier 2` del modelo 2 es: -21.625351701262645 = 55.138974309232566 - (76.76432601049524) del modelo 1
* `City_Tier 3` del modelo 2 es: -208.66762633022344 = -131.9033003197279 - (76.76432601049524) del modelo 1

Lo anterior muestra la equivalencia de los modelos pues solo se han redistribuido los coeficientes.